## Importando bibliotecas

In [45]:
import time
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException

## Configurações Iniciais

In [161]:
data_de_hoje = datetime.date.today()
data_de_hoje = data_de_hoje.strftime('%d/%m/%Y')

#### Coleta o dado 
driver = webdriver.Chrome(service=ChromeService(
    ChromeDriverManager().install()))

svg="https://www.savegnago.com.br/"
driver.get(svg)
PRODUTOS = []

try:
    rejeitar_cookies = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "cc-deny"))
    )
    rejeitar_cookies.click()
    time.sleep(1)  # Dá tempo para o banner sumir
except TimeoutException:
    print("Botão de rejeitar cookies não apareceu — seguindo em frente.")

wait = WebDriverWait(driver, 3)

try:
    # Tenta localizar pelo ID
    input_busca = wait.until(EC.element_to_be_clickable((By.ID, "downshift-1-input")))
except TimeoutException:
    # Fallback: usa o placeholder
    input_busca = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='O que você deseja?']")))

input_busca.clear()
input_busca.send_keys("salmao")
input_busca.send_keys(Keys.ENTER)

# Coleta da lista de lojas

In [ ]:
# Espera o botão aparecer e ficar clicável
botao_retire_em = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME,
    "savegnagoio-store-theme-15-x-regionalizationStoreNameButtonMob"
)))
botao_retire_em.click()
time.sleep(2)

# Aguarda e clica no botão "Retirar na loja"
botao_retirar = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME, "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
botao_retirar.click()
time.sleep(2)

In [42]:
from bs4 import BeautifulSoup
import pandas as pd

def extrair_lojas_savegnago(driver):
    """
    Extrai lista de lojas Savegnago com as colunas: cidade, bairro, endereco.
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    blocos = soup.find_all('div', class_='savegnagoio-store-theme-15-x-containerCardStageThree')

    lojas = []

    for bloco in blocos:
        nome_tag = bloco.find('p', class_='savegnagoio-store-theme-15-x-nameCardStageThree')
        endereco_tags = bloco.find_all('p', class_='savegnagoio-store-theme-15-x-addressCardStageThree')

        if nome_tag and len(endereco_tags) >= 1:
            nome_loja = nome_tag.text.strip()
            endereco = endereco_tags[0].text.strip()

            # Lógica aprimorada para separar cidade e bairro
            if " - Bairro " in nome_loja:
                cidade, bairro = nome_loja.split(" - Bairro ", 1)
            elif nome_loja.count(" - ") == 1:
                cidade, bairro = nome_loja.split(" - ", 1)
            else:
                cidade = nome_loja.strip()
                bairro = ""

            lojas.append({
                "cidade": cidade.strip(),
                "bairro": bairro.strip(),
                "endereco": endereco.strip()
            })

    return pd.DataFrame(lojas)


In [44]:
# ... (depois de clicar em "Retirar na loja" e garantir que a lista esteja visível)
df_lojas = extrair_lojas_savegnago(driver)
df_lojas.to_csv("savegnago_lojas.csv", index=False)
df_lojas

,cidade,bairro,endereco
0,Americana,Centro,"Rua Trinta de Julho , 477"
1,Araraquara,Centro,"Avenida Padre Francisco Sales Colturato , 1600"
2,Araraquara,Vila Sedenho,"Rua Maurício Galli , 555"
3,Araraquara,Via Expressa,"Avenida Maria Antonia Camargo de Oliveira , 1030"
4,Araras,Vila Michelin,"Avenida Dona Renata , 3605"
5,Barretos,Jardim América,Avenida Engenheiro Necker Carvalho de Camargo ...
6,Bebedouro,Jardim do Sonho,"Avenida Quito Stamato , 555"
7,Campinas,Botafogo,"Rua Mário Siqueira , 221"
8,Campinas,Conj. Hab. Pe. Anchieta,"Rua Dom Aloisio Lorscheider , 225"
9,Campinas,Jardim do Lago,"Avenida das Amoreiras , 3033"


# Fechamento lista de lojas

In [186]:
##### Botao: Retire em
try:
    # Aguarda botão com texto "Retire em:"
    botao_retire_em = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//p[contains(text(), 'Retire em:')]"))
    )

    # Sobe ao botão pai (button) e clica
    driver.execute_script("arguments[0].scrollIntoView(true);", botao_retire_em)
    botao_retire_em.find_element(By.XPATH, "./ancestor::button").click()

except TimeoutException:
    print("❌ Botão 'Retire em:' não encontrado ou não clicável.")

time.sleep(2)

#### Aguarda e clica no botão "Retirar na loja"
botao_retirar = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME, "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
botao_retirar.click()
time.sleep(2)


In [187]:
# Aguarda a lista de lojas
lojas = wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree"))
)
len(lojas)

41

In [193]:
##### Botao: Retire em
try:
    # Aguarda botão com texto "Retire em:"
    botao_retire_em = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.XPATH, "//p[contains(text(), 'Retire em:')]"))
    )

    # Sobe ao botão pai (button) e clica
    driver.execute_script("arguments[0].scrollIntoView(true);", botao_retire_em)
    botao_pai = botao_retire_em.find_element(By.XPATH, "./ancestor::button")
    driver.execute_script("arguments[0].click();", botao_pai)


except TimeoutException:
    print("❌ Botão 'Retire em:' não encontrado ou não clicável.")

time.sleep(2)

#### Aguarda e clica no botão "Retirar na loja"
botao_retirar = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME, "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
botao_retirar.click()
time.sleep(2)


# Aguarda a lista de lojas
lojas = wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree"))
)

# Trabalha apenas com a primeira loja
primeira_loja = lojas[30]

# Faz scroll até a primeira loja
driver.execute_script("arguments[0].scrollIntoView();", primeira_loja)
time.sleep(1)

# Clica na primeira loja
primeira_loja.click()

In [194]:
from selenium.webdriver.common.action_chains import ActionChains

# === CLICA NO BOTÃO "Ir às compras" COM O MOUSE ===
botao_ir_compras = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME,
    "savegnagoio-store-theme-15-x-buttonRegionalization"
)))

# Usa ActionChains para clicar como se fosse o mouse
actions = ActionChains(driver)
actions.move_to_element(botao_ir_compras).pause(0.5).click().perform()
time.sleep(2)

# === CLICA NO X PARA FECHAR O MODAL (se aparecer) ===
try:
    fechar_modal = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "savegnagoio-store-theme-15-x-iconClose"))
    )
    actions.move_to_element(fechar_modal).pause(0.5).click().perform()
    time.sleep(1)
except TimeoutException:
    print("✅ Nenhum modal para fechar.")


In [167]:
# Aguarda o botão "Ir às compras" e clica
botao_ir_compras = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME,
    "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
botao_ir_compras.click()

time.sleep(2)  # pequena pausa para carregamento da nova tela


fechar_modal = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "savegnagoio-store-theme-15-x-iconClose"))
)
fechar_modal.click()


In [181]:
# Aguarda e clica no botão "Retirar na loja"
botao_retirar = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME, "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
botao_retirar.click()
time.sleep(2)

In [183]:
# Aguarda o botão e clica via JavaScript
botao_ir_compras = wait.until(EC.element_to_be_clickable((
    By.CLASS_NAME,
    "savegnagoio-store-theme-15-x-buttonRegionalization"
)))
driver.execute_script("arguments[0].scrollIntoView();", botao_ir_compras)
time.sleep(1)
driver.execute_script("arguments[0].click();", botao_ir_compras)


In [182]:


# Aguarda a lista de lojas
lojas = wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree"))
)


# Trabalha apenas com a primeira loja
primeira_loja = lojas[15]

# Faz scroll até a primeira loja
driver.execute_script("arguments[0].scrollIntoView();", primeira_loja)
time.sleep(1)

# Clica na primeira loja
primeira_loja.click()

In [173]:
# Aguarda e coleta os elementos de cada loja
lojas = wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree"))
)

for i in range(len(lojas)):
    # Coleta todos os elementos novamente a cada iteração, pois a página pode recarregar
    lojas = driver.find_elements(By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree")
    
    # Scroll até a loja desejada (evita erro de elemento fora da tela)
    driver.execute_script("arguments[0].scrollIntoView();", lojas[i])
    time.sleep(1)
    
    # Clica na loja
    lojas[i].click()
    
    wait = WebDriverWait(driver, 1)

    botao_ir_compras = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((
            By.CLASS_NAME,
            "savegnagoio-store-theme-15-x-buttonRegionalization"
        ))
    )
    botao_ir_compras.click()


    

    
    # ===> AQUI VÃO OS PROCEDIMENTOS QUE VOCÊ QUER EXECUTAR DENTRO DA LOJA <===
    # Exemplo fictício:
    # produto = wait.until(EC.presence_of_element_located((By.ID, "productName")))
    # print(produto.text)
    
    time.sleep(2)  # Aguarda para garantir carregamento

    # # Volta para a lista de lojas (ajustar conforme botão ou caminho real)
    # driver.back()
    
    # # Aguarda o carregamento novamente da lista
    # wait.until(EC.presence_of_all_elements_located(
    #     (By.CLASS_NAME, "savegnagoio-store-theme-15-x-listCardStageThree"))
    # )

# Encerra
driver.quit()


IndexError: list index out of range

## Savegnago

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.122)
Stacktrace:
0   chromedriver                        0x000000010468dd14 cxxbridge1$str$ptr + 2735276
1   chromedriver                        0x0000000104685f88 cxxbridge1$str$ptr + 2703136
2   chromedriver                        0x00000001041d66f0 cxxbridge1$string$len + 90424
3   chromedriver                        0x00000001041b0720 chromedriver + 132896
4   chromedriver                        0x0000000104245cc4 cxxbridge1$string$len + 546572
5   chromedriver                        0x000000010425ec08 cxxbridge1$string$len + 648784
6   chromedriver                        0x0000000104211be8 cxxbridge1$string$len + 333360
7   chromedriver                        0x0000000104651598 cxxbridge1$str$ptr + 2487600
8   chromedriver                        0x0000000104654830 cxxbridge1$str$ptr + 2500552
9   chromedriver                        

In [ ]:
# Espera até o botão de seleção da loja estar presente e clicável
wait = WebDriverWait(driver, 10)
botao_loja = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,
    "button.savegnagoio-store-theme-15-x-regionalizationStoreNameButton")))
botao_loja.click()

time.sleep(2)  # Pequena pausa para abrir o modal


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010468dd14 cxxbridge1$str$ptr + 2735276
1   chromedriver                        0x0000000104685f88 cxxbridge1$str$ptr + 2703136
2   chromedriver                        0x00000001041d66f0 cxxbridge1$string$len + 90424
3   chromedriver                        0x000000010421d9e0 cxxbridge1$string$len + 381992
4   chromedriver                        0x000000010425f0c8 cxxbridge1$string$len + 650000
5   chromedriver                        0x0000000104211be8 cxxbridge1$string$len + 333360
6   chromedriver                        0x0000000104651598 cxxbridge1$str$ptr + 2487600
7   chromedriver                        0x0000000104654830 cxxbridge1$str$ptr + 2500552
8   chromedriver                        0x0000000104631c14 cxxbridge1$str$ptr + 2358188
9   chromedriver                        0x00000001046550b8 cxxbridge1$str$ptr + 2502736
10  chromedriver                        0x0000000104622dec cxxbridge1$str

In [ ]:
retirada = driver.find_element(By.CLASS_NAME, "savegnagoio-store-theme-9-x-buttonRegionalization")
retirada.click()

In [ ]:
lojas = driver.find_elements(By.CLASS_NAME,"savegnagoio-store-theme-9-x-containerCardStageThree")
nome_loja, endereco_loja, _ = lojas[0].text.split("\n")

lojas[0].click()


In [ ]:
ir = driver.find_elements(By.CLASS_NAME, "savegnagoio-store-theme-9-x-buttonRegionalization")
ir[1].click()

In [ ]:
pesquisa = driver.find_element(By.TAG_NAME, "input")
pesquisa.send_keys("picanha")
pesquisa.send_keys(Keys.ENTER)

In [ ]:
prods = driver.find_elements(By.CLASS_NAME, "vtex-search-result-3-x-galleryItem.vtex-search-result-3-x-galleryItem--normal.vtex-search-result-3-x-galleryItem--grid.pa4")

In [ ]:
for prod in prods:
    PRODUTOS.append(prod.text + "|" + nome_loja + "|" + endereco_loja )

In [ ]:
for i in range(1, len(lojas)):
    info_cep = driver.find_element(By.CLASS_NAME, "savegnagoio-store-theme-9-x-regionalizationStoreNameButton.regionalizationStoreButton")
    info_cep.click()
    time.sleep(2)
    retirada = driver.find_element(By.CLASS_NAME, "savegnagoio-store-theme-9-x-buttonRegionalization")
    retirada.click()
    time.sleep(2)
    loja = driver.find_elements(By.CLASS_NAME,"savegnagoio-store-theme-9-x-containerCardStageThree")[i]
    nome_loja, endereco_loja, _ = loja.text.split("\n")
    ActionChains(driver).scroll_to_element(loja).perform()
    loja.click()
    time.sleep(2)
    if driver.find_element(By.XPATH, "//*[contains(text(), 'Não é possível fazer a retirada na loja selecionada!')]").text == 'Não é possível fazer a retirada na loja selecionada!':
        driver.find_element(By.ID, "icon_x").click()
        continue
    ir = driver.find_elements(By.CLASS_NAME, "savegnagoio-store-theme-9-x-buttonRegionalization")
    ir[1].click()
    
    time.sleep(1)
    pesquisa = driver.find_element(By.TAG_NAME, "input")
    pesquisa.send_keys("picanha")
    pesquisa.send_keys(Keys.ENTER)
    time.sleep(5)
    prods= driver.find_elements(By.CLASS_NAME, "vtex-search-result-3-x-galleryItem.vtex-search-result-3-x-galleryItem--normal.vtex-search-result-3-x-galleryItem--grid.pa4")
    for prod in prods:
        PRODUTOS.append(prod.text + "|" + nome_loja + "|" + endereco_loja )

In [ ]:
## produto, preco, desconto, 
LIST_TO_DF = []
for prod in PRODUTOS:
    nome_prod, loja, endereco = prod.split("|")
    cidade, _ = loja.split(" - ")
    if nome_prod.endswith("Produto indisponível"): 
        continue
    else:
        nome_prod = nome_prod.split("\n")
        nome = nome_prod[0]
        desconto = "NaN"
        if (len(nome_prod)) == 5:
            if nome_prod[1].endswith("/Kg"):
                preco = "R$" + nome_prod[1].split("R$")[1]
            else:
                preco = nome_prod[1]
            
        if (len(nome_prod)) == 7:
            desconto = nome_prod[2]
            if nome_prod[3].endswith("/Kg"):
                preco = "R$" + nome_prod[3].split("R$")[1]
            else:
                preco = nome_prod[3]

    LIST_TO_DF.append([nome, preco, desconto, endereco, cidade])        

In [ ]:
produtos_df = pd.DataFrame(LIST_TO_DF, columns=["Produto", "Preco",  "Desconto", "Endereco", "Cidade"])
produtos_df["Cidade"]

,Produto,Preco,Desconto,Endereco,Cidade
0,Macarrão Instantâneo Renata 88g Lámen Cremoso ...,"R$ 1,99",NaN,"Rua Trinta de Julho , 477",Americana
1,Hamburguer Brasa Burguers 672g Picanha,"R$ 16,95",20%,"Rua Trinta de Julho , 477",Americana
2,Hamburguer Brasa Burguers 56g Picanha,"R$ 1,69",NaN,"Rua Trinta de Julho , 477",Americana
3,Sanduíche Hot Pocket Sadia X-Picanha 145g,"R$ 10,70",NaN,"Rua Trinta de Julho , 477",Americana
4,Salgadinho Elma Chips Pingo D'Ouro 90g Picanha,"R$ 7,89",NaN,"Rua Trinta de Julho , 477",Americana
...,...,...,...,...,...
232,"Caldo Pó Sazon 32,5g Picanha","R$ 2,99",NaN,"Rua Joaquim Evangelista de Tolêdo , 243",São Carlos
233,Alimento Cao Menu Chefe Baw Waw 50g Bifinho Pi...,"R$ 3,98",NaN,"Rua Joaquim Evangelista de Tolêdo , 243",São Carlos
234,Hamburguer Brasa Burguers 300g Picanha,"R$ 13,20",29%,"Rua Joaquim Evangelista de Tolêdo , 243",São Carlos
235,Salgadinho Elma Chips Pingo Douro Picanha 55g,"R$ 5,15",NaN,"Rua Joaquim Evangelista de Tolêdo , 243",São Carlos


In [ ]:
produtos_df["Cidade"].unique()

array(['Americana', 'Campinas', 'Franca', 'Hortolândia', 'Jaboticabal',
       'Jardinópolis', 'Piracicaba', 'Ribeirão Preto', 'Rio Claro',
       'Sertãozinho', 'Sumaré', 'São Carlos'], dtype=object)

## Carrefour

### Função de Coleta

In [ ]:
#### para eliminar linhas com erro
def excluir_linha (dados, a):
    dados = dados.drop(dados.index[[a]])
    return dados

def coleta_precos_carrefour(divs_carrefour):
    precos_carrefour = []
    for div in divs_carrefour:
        precos_carrefour.append(div.text)
        
    for i in range(len(precos_carrefour)):
        precos = precos_carrefour[i].split("\n")
        if len(precos) == 2:
            precos_carrefour[i] = precos
        elif len(precos) == 3:
            if precos[0].startswith("LEVE"):
                precos_carrefour[i] = precos[1:]
            else:
                precos_carrefour[i] = precos[0:2]
        elif len(precos) == 4:
            if precos[1][-1]=="1":
                temp = precos.copy()
                precos = [temp[2], temp[3]]
            elif precos[2][-1]=="%":
                temp = precos.copy()
                precos = [temp[0], temp[3], temp[2]]
            elif precos[0][-1]=="F":
                temp = precos.copy()
                precos = [temp[2], temp[3], temp[0]]
            elif precos[0][-1]=="N":
                temp = precos.copy()
                precos = [temp[2], temp[3], temp[0]]
            precos_carrefour[i] = precos
        elif len(precos) == 5:
            precos.pop(2)
            precos.pop(0)
            precos_carrefour[i] = precos
        elif len(precos) ==6:
            temp = precos.copy()
            precos = [temp[2], temp[5], temp[4]]
            precos_carrefour[i] = precos

    precos_carrefour = pd.DataFrame(precos_carrefour)
    precos_carrefour['data'] = data_de_hoje

    cols = list(precos_carrefour)
    cols.insert(0, cols.pop(cols.index('data')))
    precos_carrefour = precos_carrefour.loc[:, cols]

    precos_carrefour = precos_carrefour.rename(columns={'data':'data',
                                            0:'produto',
                                            1: 'preco',
                                            2:'desconto'
                                            })

    precos_carrefour['Vendedor'] = 'Carrefour'

    precos_carrefour = precos_carrefour.drop((precos_carrefour.loc[(precos_carrefour['preco']=="Produto Indisponível")]).index)
    precos_carrefour = precos_carrefour.dropna(thresh=3)

    return precos_carrefour

### Vinho

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=vinhos&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
driver.find_element(By.CSS_SELECTOR, "div[aria-label='Retire na Loja']").click()
select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))

cidades = []
for i in range(len(select.options)):
    cidades.append(select.options[i].text)
index_brasilia = cidades.index('BRASILIA - DF')

select.select_by_index(index_brasilia)

botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
loja, _, cidade, _ = botao_cidade[0].text.split("\n")
botao_cidade[0].click()
time.sleep(4)

In [ ]:
qdte_prod = driver.find_element(By.CSS_SELECTOR, '#__next > main > section.container.max-sm\:\[\&_\.carrousel-produtos-patrocinados\>div\]\:m-0.max-sm\:\[\&_\.sponsored-products-arrow-right\]\:hidden.max-sm\:\[\&_\.sponsored-products-arrow-left\]\:hidden.max-lg\:\[\&_\.sponsored-shelf_\.add-to-cart-button\]\:bg-\[\#e81f25\].max-lg\:\[\&_\.sponsored-shelf_\.swiper-pagination-bullet-active\]\:bg-\[\#e81f25\].max-lg\:\[\&_\.sponsored-products-arrow-left\>svg\>path\]\:stroke-\[\#e81f25\].max-lg\:\[\&_\.sponsored-products-arrow-right\>svg\>path\]\:stroke-\[\#e81f25\] > div.flex.gap-12.justify-center.lg\:justify-start > div.flex.flex-col.lg\:gap-5.w-full.max-w-full.lg\:max-w-\[73\.3\%\] > div:nth-child(4) > div.flex.flex-col.justify-between.py-\[30px\].md\:flex-row > div.flex.justify-center.mt-\[30px\].md\:m-0 > div > button > span')
qdte_prod.click()

In [ ]:
qtde_botao = driver.find_element(By.CSS_SELECTOR, '#__next > main > section.container.max-sm\:\[\&_\.carrousel-produtos-patrocinados\>div\]\:m-0.max-sm\:\[\&_\.sponsored-products-arrow-right\]\:hidden.max-sm\:\[\&_\.sponsored-products-arrow-left\]\:hidden.max-lg\:\[\&_\.sponsored-shelf_\.add-to-cart-button\]\:bg-\[\#e81f25\].max-lg\:\[\&_\.sponsored-shelf_\.swiper-pagination-bullet-active\]\:bg-\[\#e81f25\].max-lg\:\[\&_\.sponsored-products-arrow-left\>svg\>path\]\:stroke-\[\#e81f25\].max-lg\:\[\&_\.sponsored-products-arrow-right\>svg\>path\]\:stroke-\[\#e81f25\] > div.flex.gap-12.justify-center.lg\:justify-start > div.flex.flex-col.lg\:gap-5.w-full.max-w-full.lg\:max-w-\[73\.3\%\] > div:nth-child(4) > div.flex.flex-col.justify-between.py-\[30px\].md\:flex-row > div.flex.justify-center.mt-\[30px\].md\:m-0 > div > div > button:nth-child(5)')
qtde_botao.click()
time.sleep(4)
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_vinho = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_vinho

,data,produto,preco,desconto,Vendedor
0,31/01/2024,Vinho Tinto Casa Crebar 2018 Cabernet Sauvigno...,"R$ 26,90",50% OFF NA 2 UN,Carrefour
1,31/01/2024,Vinho Tinto Chileno Concha Y Toro Reservado Ca...,"R$ 40,99",50% OFF NA 2 UN,Carrefour
2,31/01/2024,Vinho Tinto Chileno Casillero del Diablo Caber...,"R$ 65,99",50% OFF NA 2 UN,Carrefour
3,31/01/2024,Vinho Branco Seco Casal Garcia Premium 750ml,"R$ 56,99",50% OFF NA 2 UN,Carrefour
4,31/01/2024,Vinho Tinto Chileno Concha Y Toro Reservado Ca...,"R$ 32,89",50% OFF NA 2 UN,Carrefour
5,31/01/2024,Vinho Branco Seco Pardal Telhado 2019 750 ml,"R$ 69,99",50% OFF NA 2 UN,Carrefour
6,31/01/2024,Vinho Tinto Chileno Casillero del Diablo Carme...,"R$ 65,99",50% OFF NA 2 UN,Carrefour
7,31/01/2024,Vinho Tinto 8Km Cabernet Sauvignon 750ml,"R$ 29,90",50% OFF NA 2 UN,Carrefour
8,31/01/2024,Vinho Tinto Chileno Merlot Casa Crebar - 750 ml,"R$ 26,90",50% OFF NA 2 UN,Carrefour
9,31/01/2024,Vinho Tinto Pardal Telhado 2019 750 ml,"R$ 69,99",50% OFF NA 2 UN,Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_vinho = []
# for div in divs_carrefour:
#     precos_carrefour_vinho.append(div.text)

# precos_carrefour_vinho

### Azeite

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=azeites&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_azeite = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_azeite

,data,produto,preco,Vendedor
0,31/01/2024,Azeite Chileno Extra Virgem O-Live 500ml,"R$ 35,39",Carrefour
1,31/01/2024,Azeite Terras de Camões Extra Virgem Frutado 5...,"R$ 33,09",Carrefour
2,31/01/2024,Azeite Extra Virgem PTG Do Monte 500ml,"R$ 32,29",Carrefour
3,31/01/2024,Azeite de Oliva Português Extra Virgem Gallo G...,"R$ 35,39",Carrefour
4,31/01/2024,Azeite Andorinha Extra Virgem 500ml,"R$ 44,29",Carrefour
5,31/01/2024,Azeite Português Extra Virgem Tradicional Gall...,"R$ 48,99",Carrefour
6,31/01/2024,Azeite Espanhol Extra Virgem Carrefour 500ml,"R$ 30,90",Carrefour
7,31/01/2024,Azeite Português Tradicional Gallo Tipo Único ...,"R$ 44,29",Carrefour
8,31/01/2024,Azeite do Chile Extra Virgem O-l!ve 1L,"R$ 54,90",Carrefour
9,31/01/2024,Azeite Espanhol Extra Virgem Carbonell 500ml,"R$ 38,99",Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_azeite = []
# for div in divs_carrefour:
#     precos_carrefour_azeite.append(div.text)

# precos_carrefour_azeite

### Tilapia

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=tilapia&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_tilapia = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_tilapia

,data,produto,preco,desconto,Vendedor
0,31/01/2024,Filé de Tilápia Congelado Copacol 800g,"R$ 44,90",-21%,Carrefour
1,31/01/2024,Filé de Tilápia Resfriada Carrefour Aprox. 900g,"R$ 55,16",None,Carrefour
2,31/01/2024,Filé de Tilápia Resfriada Carrefour Aprox. 400g,"R$ 27,07",None,Carrefour
3,31/01/2024,Filé de Tilápia Resfriada Carrefour Aprox. 250g,"R$ 16,92",None,Carrefour
4,31/01/2024,Filé de Tilápia a Milanesa Congelado Swift 500 g,"R$ 43,96",None,Carrefour
5,31/01/2024,Filé de Tilápia in Natura Congelado 400g,"R$ 35,49",None,Carrefour
6,31/01/2024,Filé de Tilápia Empanado Congelado Multigrãos ...,"R$ 32,79",None,Carrefour
7,31/01/2024,Tilápia Fresca Inteira Limpa Carrefour Aprox. ...,"R$ 19,67",None,Carrefour
8,31/01/2024,Filé de Tilápia Empanado Congelado Copacol 400g,"R$ 32,79",None,Carrefour
9,31/01/2024,Filé de Tilápia Empanado Congelado Corn Flakes...,"R$ 40,69",None,Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_tilapia = []
# for div in divs_carrefour:
#     precos_carrefour_tilapia.append(div.text)

# precos_carrefour_tilapia

### Sardinha

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=sardinha&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_sardinha = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_sardinha 

,data,produto,preco,Vendedor
0,31/01/2024,Sardinhas Em Azeite De Oliva General 125g,"R$ 8,49",Carrefour
1,31/01/2024,Sardinha em Óleo Gomes da Costa 125g,"R$ 5,99",Carrefour
2,31/01/2024,Sardinhas Ao Molho De Tomate General 125g,"R$ 7,79",Carrefour
3,31/01/2024,Sardinha com Molho de Tomate 88 125g,"R$ 6,79",Carrefour
4,31/01/2024,Sardinha com Molho de Tomate Gomes da Costa 125g,"R$ 5,99",Carrefour
5,31/01/2024,Sardinhas Ao Molho De Tomate Picante General 125g,"R$ 7,99",Carrefour
6,31/01/2024,Sardinhas Em Óleo Comestivel General 125g,"R$ 7,39",Carrefour
7,31/01/2024,Sardinha com Molho de Tomate Gomes da Costa 250g,"R$ 11,59",Carrefour
8,31/01/2024,Sardinha Defumada em Óleo Gomes da Costa 125g,"R$ 5,99",Carrefour
9,31/01/2024,Sardinha ao Natural com Molho de Ervas Gomes d...,"R$ 5,99",Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_sardinha = []
# for div in divs_carrefour:
#     precos_carrefour_sardinha.append(div.text)

# precos_carrefour_sardinha

['50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto Casa Crebar 2018 Cabernet Sauvignon 750ml\nR$ 26,90',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto Chileno Concha Y Toro Reservado Cabernet Sauvignon 750 ml\nR$ 40,99',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto Chileno Casillero del Diablo Cabernet Sauvignon Reserva 750 ml\nR$ 65,99',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Branco Seco Casal Garcia Premium 750ml\nR$ 56,99',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto Chileno Concha Y Toro Reservado Carménère 750 ml\nR$ 32,89',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Branco Seco Pardal Telhado 2019 750 ml\nR$ 69,99',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto Chileno Casillero del Diablo Carmenere Reserva 750 ml\nR$ 65,99',
 '50% OFF NA 2 UN\nBadge da promoção 50% OFF NA 2 UN\nVinho Tinto 8Km Cabernet Sauvignon 750ml\nR$ 29,90',
 '50% OFF NA 2 

### Salmão

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=salmao&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_sardinha = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_sardinha 

,data,produto,preco,Vendedor
0,31/01/2024,Salmão Inteiro Fresco Aprox. 4Kg,"R$ 279,60",Carrefour
1,31/01/2024,Ração Úmida Whiskas Sachê Salmão ao Molho para...,"R$ 3,29",Carrefour
2,31/01/2024,Filé de Salmão com Pele Resfriado Carrefour Ap...,"R$ 78,21",Carrefour
3,31/01/2024,Lombo Salmão Resfriado sem Pele Carrefour Apro...,"R$ 39,96",Carrefour
4,31/01/2024,Filé de Salmão Resfriado com Pele Carrefour Ap...,"R$ 34,76",Carrefour
5,31/01/2024,Lombo de Salmão Resfriado Carrefour Aprox. 250g,"R$ 24,47",Carrefour
6,31/01/2024,Filé de Salmão Resfriado sem Pele Carrefour Ap...,"R$ 39,96",Carrefour
7,31/01/2024,Ração Úmida para Cachorro Adulto Pequeno Purin...,"R$ 3,39",Carrefour
8,31/01/2024,Pedaços de Salmão com Pele Congelado Mercado C...,"R$ 114,90",Carrefour
9,31/01/2024,Filé de Salmão sem Espinha Swift Premium 900 g,"R$ 94,96",Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_salmao = []
# for div in divs_carrefour:
#     precos_carrefour_salmao.append(div.text)

# precos_carrefour_salmao

### Bacalhau

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=bacalhau&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_sardinha = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_sardinha 

,data,produto,preco,Vendedor
0,31/01/2024,Lombo de Bacalhau sem Espinha Swift 1Kg,"R$ 119,97",Carrefour
1,31/01/2024,Bacalhau Gadus Morhua Salgado Desfiado Resfria...,"R$ 33,97",Carrefour
2,31/01/2024,Bacalhau Porto Pedaço Resfriado Carrefour Apro...,"R$ 151,08",Carrefour
3,31/01/2024,Bacalhau Porto Desfiado Morhua Codinho Carrefo...,"R$ 30,22",Carrefour
4,31/01/2024,Bacalhau Gadus Morhua Salgado Lombo Resfriado ...,"R$ 152,01",Carrefour
5,31/01/2024,Filé de Bacalhau Salgado Macrocephalus Carrefo...,"R$ 152,91",Carrefour
6,31/01/2024,Bacalhau Gadus Morhua Salgado Borboleta Resfri...,"R$ 152,01",Carrefour
7,31/01/2024,Bacalhau Ling Salgado Carrefour Aproximadament...,"R$ 73,71",Carrefour
8,31/01/2024,Bacalhau Zarbo Salgado Desfiado Carrefour Apro...,"R$ 20,97",Carrefour
9,31/01/2024,Bacalhau Gadus Morhua Salgado Posta Resfriado ...,"R$ 128,61",Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_bacalhau = []
# for div in divs_carrefour:
#     precos_carrefour_bacalhau.append(div.text)

# precos_carrefour_bacalhau

### Atum

In [ ]:
carrefour = "https://mercado.carrefour.com.br/s?q=atum&sort=score_desc&page=0"
driver.get(carrefour)
time.sleep(4)

In [ ]:
divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')
precos_carrefour_sardinha = coleta_precos_carrefour(divs_carrefour)
precos_carrefour_sardinha 

,data,produto,preco,desconto,Vendedor
0,31/01/2024,Atum Sólido Natural Carrefour 170 g,"R$ 7,80",-11%,Carrefour
1,31/01/2024,Atum Ralado Natural Carrefour 170 g,"R$ 5,59",None,Carrefour
2,31/01/2024,Atum Sólido Em Óleo Comestível 170 g,"R$ 8,69",None,Carrefour
3,31/01/2024,Atum Ralado Em Óleo Comestível 170 g,"R$ 5,31",-5%,Carrefour
4,31/01/2024,Atum Sólido ao Natural Gomes da Costa 170g,"R$ 8,89",None,Carrefour
5,31/01/2024,Atum Ralado ao Natural Gomes da Costa 170g,"R$ 7,69",None,Carrefour
6,31/01/2024,Atum Ralado em Óleo Gomes da Costa 170g,"R$ 7,69",None,Carrefour
7,31/01/2024,Atum em Óleo Pedaços Gomes da Costa 170g,"R$ 11,99",None,Carrefour
8,31/01/2024,Atum ao Natural Pedaços Gomes da Costa 170g,"R$ 11,99",None,Carrefour
9,31/01/2024,Atum Defumado Sólido em Óleo Gomes da Costa 170g,"R$ 8,89",None,Carrefour


In [ ]:
# divs_carrefour = driver.find_elements(By.CSS_SELECTOR, 'section.flex-1.flex.flex-col')

# precos_carrefour_atum = []
# for div in divs_carrefour:
#     precos_carrefour_atum.append(div.text)


# precos_carrefour_atum

## Pão de Açucar

### Função de Coleta

In [ ]:
def coleta_precos_pda(driver):
    css_class = "product-cardstyles__CardStyled-sc-1uwpde0-0"
    produtos_pda = driver.find_elements(By.CLASS_NAME, css_class)
    #precos_pda2 = [produto.text.split("\n") for produto in produtos_pda]
    precos_pda2 = []
    lista = ['Cães', 'Cao', 'Gatos', 'gato', 'Gato', 'Racao', 'Brinquedo', 'PURINA', 'KELCAT', 'OCP']

    for produto in produtos_pda:
        p = produto.text.split("\n")
        #print(len(p))
        if len(p)>3:
            temp=p.copy()
            if temp[3].endswith(" OFF"):
                p=[temp[0], temp[2], "-"+temp[3].rstrip(" OFF")]
            else:
                if temp[2].endswith(" OFF"):
                    p=[temp[0],temp[3].lstrip('A unid. sai por'), "-"+temp[2].rstrip(" OFF")]
                else:
                    p=[temp[0],temp[1]]
        if "Vendido por:" not in p:
            for item in lista:
                if item in p[0]:
                    p =[]
                    break
            if len(p)>0:
                precos_pda2.append(p)


    precos_pda2 = pd.DataFrame(precos_pda2)
    precos_pda2['data'] = data_de_hoje

    cols = list(precos_pda2)
    cols.insert(0, cols.pop(cols.index('data')))
    precos_pda2 = precos_pda2.loc[:, cols]

    precos_pda2['Vendedor'] = "Pão de Açucar"

    precos_pda2 = precos_pda2.rename(columns={'data':'data',
                                            0:'produto',
                                            1: 'preco',
                                            2: 'desconto'
                                            })

    precos_pda2 = precos_pda2.drop((precos_pda2.loc[(precos_pda2['preco']=="INDISPONÍVEL")]).index)
    #precos_pda2 = precos_pda2.drop((precos_pda2.loc[(precos_pda2.produto.str.contains('Gatos', regex=False))]).index)
    #precos_pda2 = precos_pda2.drop((precos_pda2.loc[(precos_pda2.produto.str.contains('Cao', regex=False))]).index)
    precos_pda2 = precos_pda2.replace("COMPRAR", None)
    return precos_pda2

### Tilapia

In [ ]:
pda="https://www.paodeacucar.com"
driver.get(pda)

In [ ]:
search_box = driver.find_element(By.TAG_NAME, "input")
search_box.send_keys(Keys.CONTROL + "a")
search_box.send_keys(Keys.DELETE)
search_box.send_keys("tilapia")
search_box.send_keys(Keys.ENTER)
time.sleep(4)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
preco_pda_tilapia = coleta_precos_pda(driver)
preco_pda_tilapia

,data,produto,preco,desconto,Vendedor
0,31/01/2024,Filé De Tilápia Fresco 500g,"R$30,56",-15%,Pão de Açucar
1,31/01/2024,File de Tilapia Congelado Taeq 500gr,"R$26,94",-40%,Pão de Açucar
2,31/01/2024,Filé de Tilápia sem Pele Swift 800g,"R$44,90",None,Pão de Açucar
3,31/01/2024,Mini Combinado De Salmão 10 peças,"R$27,20",-20%,Pão de Açucar
4,31/01/2024,Hossomaki De Salmão Bandeja 12 peças,"R$19,20",-20%,Pão de Açucar
5,31/01/2024,Sashimi De Salmão Mini 9 fatias,"R$22,40",-20%,Pão de Açucar
6,31/01/2024,Niguiri De Salmão 6 peças,"R$20,80",-20%,Pão de Açucar
7,31/01/2024,"Salmão do Atlântico 4,5Kg","R$270,95",-10%,Pão de Açucar
8,31/01/2024,Tilápia em Filé Sustentável Korin 450g,"R$60,99",None,Pão de Açucar
9,31/01/2024,Salmão Defumado Fatiado DAMM 80g,"R$25,99",None,Pão de Açucar


### Salmão

In [ ]:
search_box.send_keys(Keys.CONTROL + "a")
search_box.send_keys(Keys.DELETE)
search_box.send_keys("salmão")
search_box.send_keys(Keys.ENTER)
time.sleep(4)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
preco_pda_salmao = coleta_precos_pda(driver)
preco_pda_salmao

,data,produto,preco,desconto,Vendedor
0,31/01/2024,Filé De Tilápia Fresco 500g,"R$30,56",-15%,Pão de Açucar
1,31/01/2024,"Salmão do Atlântico 4,5Kg","R$270,95",-10%,Pão de Açucar
2,31/01/2024,Hossomaki De Salmão Bandeja 12 peças,"R$19,20",-20%,Pão de Açucar
3,31/01/2024,Sashimi De Salmão Mini 9 fatias,"R$22,40",-20%,Pão de Açucar
4,31/01/2024,Niguiri De Salmão 6 peças,"R$20,80",-20%,Pão de Açucar
5,31/01/2024,Mini Combinado De Salmão 10 peças,"R$27,20",-20%,Pão de Açucar
6,31/01/2024,Salmão Porcionado SWIFT 125g,"R$15,98",None,Pão de Açucar
7,31/01/2024,Filé de Salmão Premium SWIFT 900g,"R$94,98",None,Pão de Açucar
8,31/01/2024,Salmão Fatiado QUALITÁ 100g,"R$21,90",-12%,Pão de Açucar
9,31/01/2024,Salmão Defumado Fatiado DAMM 80g,"R$25,99",None,Pão de Açucar


## Adicionando dados às planilhas